<a href="https://colab.research.google.com/github/cjfghk5697/Practice_Pytorch/blob/main/Practice_pytorch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
learning_rate = 1e-3 #얼마만큼 학습할지, 기울기를 보고 하산을 함. 실제로 얼마만큼 내려올지 learning_rate가 적으면 조금 내려오고 보고 다시하는 것
#값 설정은 성공했던 값을 가져와서 해야한다.
batch_size = 64 #미니배치 학습
epochs = 5 #몇번 다시 반복할지

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #loss 정해지고 하는것
#SGD 외에도 있다. 다양함. 모델 파라미터와 learning_rate를 결정함.


In [7]:
def train_loop(dataloader, model, loss_fn, optimizer): 
    size = len(dataloader.dataset) #데이터셋 사이즈 가져옴
    for batch, (X, y) in enumerate(dataloader): #batch 사이즈만큼 x,y를 가져옴
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad() #optimizer를 zero상태로 만듦.
        loss.backward() #미분값 다 계산
        optimizer.step() # 업데이트

        if batch % 100 == 0: #100번에 한번씩
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
# 

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() #맞

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.293261  [    0/60000]
loss: 2.289514  [ 6400/60000]
loss: 2.276023  [12800/60000]
loss: 2.279944  [19200/60000]
loss: 2.263238  [25600/60000]
loss: 2.236285  [32000/60000]
loss: 2.247068  [38400/60000]
loss: 2.220452  [44800/60000]
loss: 2.215097  [51200/60000]
loss: 2.202484  [57600/60000]
Test Error: 
 Accuracy: 44.0%, Avg loss: 2.202575 

Epoch 2
-------------------------------
loss: 2.201030  [    0/60000]
loss: 2.193566  [ 6400/60000]
loss: 2.163368  [12800/60000]
loss: 2.188229  [19200/60000]
loss: 2.144526  [25600/60000]
loss: 2.093808  [32000/60000]
loss: 2.131710  [38400/60000]
loss: 2.070653  [44800/60000]
loss: 2.066625  [51200/60000]
loss: 2.039917  [57600/60000]
Test Error: 
 Accuracy: 50.4%, Avg loss: 2.038288 

Epoch 3
-------------------------------
loss: 2.046923  [    0/60000]
loss: 2.024770  [ 6400/60000]
loss: 1.957919  [12800/60000]
loss: 2.004930  [19200/60000]
loss: 1.903878  [25600/60000]
loss: 1.836928  [32000/600

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth') #저장하는 기능